<a href="https://colab.research.google.com/github/ocxh/habit-tracker/blob/master/Titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd

#데이터 읽어오기
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

#데이터 프레임 확인
#train.head()
#test.head()
#train.describe()

In [ ]:
#누락된 값 확인(훈련 데이터)
print("train.isnull().sum")
train.isnull().sum()

train.isnull().sum


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
#누락된 값 확인(테스트 데이터)
print("test.isnull().sum()")
test.isnull().sum()

test.isnull().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [8]:
#전처리(Sex,Age, Embarked, Name, Cabin, Fare, SibSp, Parch, Ticket, Pclass)(9)  not(PassengerId, Survived)(2)

#Sex
#성별을 숫자로 매핑
sex_map = {"male": 0, "female": 1}
train['Sex'] = train['Sex'].map(sex_map)
test['Sex'] = test['Sex'].map(sex_map)

#Age(추가 시 점수 떨어짐)
#누락된 값 평균으로 채우기
train["Age"] = train["Age"].fillna(train["Age"].mean())
test["Age"] = test["Age"].fillna(test["Age"].mean())

#Pclass
#one hot encoding 적용 하나 안하나 점수차이 X
#one-hot encoding 적용
'''
train['Pclass_3']=(train['Pclass']==3)
train['Pclass_2']=(train['Pclass']==2)
train['Pclass_1']=(train['Pclass']==1)

test['Pclass_3']=(test['Pclass']==3)
test['Pclass_2']=(test['Pclass']==2)
test['Pclass_1']=(test['Pclass']==1)

train=train.drop(columns='Pclass')
test=test.drop(columns='Pclass')
'''

#Fare(추가 시 점수 떨어짐)
#누락된 값 채우기 
train["Fare"] = train["Fare"].fillna(train["Fare"].mean())
test["Fare"] = test["Fare"].fillna(test["Fare"].mean())


#Family (새로운 칼럼, 있으나 없으나 점수차이 X) => (Solo칼럼을 만들기 위해서)
train['Family'] = 1 + train['SibSp'] + train['Parch']
test['Family'] = 1 + test['SibSp'] + test['Parch']

#Solo (새로운 칼럼, 추가 시 점수 오름)
train['Solo'] = (train['Family'] == 1)
test['Solo'] = (test['Family'] == 1)

#Parch (제거 시 점수 떨어짐, 그대로 유지)

#SibSp(있으나 없으나 관계 X)


#값 제거
#생존과 관계없는 값들 제거
train = train.drop(['Cabin','Embarked', 'PassengerId', 'Name','Ticket'],axis=1)
test = test.drop(['Cabin', 'Embarked', 'Name', 'Ticket'],axis=1)
#추가 시 점수가 떨어지기에 제거
train = train.drop(['Age','Fare'], axis=1)
test = test.drop(['Age','Fare'], axis=1)

In [ ]:
train.isnull().sum()

Survived    0
Pclass      0
Sex         0
SibSp       0
Parch       0
Ticket      0
Family      0
Solo        0
dtype: int64

In [ ]:
test.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
SibSp          0
Parch          0
Ticket         0
Family         0
Solo           0
dtype: int64

In [ ]:
#이상치 제거 => 있으나 없으나 점수차이 x
'''
age_mean = train['Age'].mean()
age_std = train['Age'].std()
indexNames = train[train['Age'] < age_mean - 3*age_std].index
train.drop(indexNames , inplace=True)
indexNames = train[train['Age'] > age_mean + 3*age_std].index
train.drop(indexNames , inplace=True)

fare_mean = train['Fare'].mean()
fare_std = train['Fare'].std()
indexNames = train[train['Fare'] < fare_mean - 3*fare_std].index
train.drop(indexNames , inplace=True)
indexNames = train[train['Fare'] > fare_mean + 3*fare_std].index
train.drop(indexNames , inplace=True)
'''

In [13]:
#train_input, train_target
train_input = train.drop(['Survived'], axis=1)
train_target = train['Survived']

In [ ]:
#로지스틱 Score: 0.77751
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()


lr.fit(train_input, train_target)

lr.coef_

predict = lr.predict(test.drop(['PassengerId'], axis=1))
rs =pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predict
})

In [10]:
#결정트리 Score: 0.77751
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(max_depth=4, random_state=42)
dt.fit(train_input, train_target)

predict = dt.predict(test.drop(['PassengerId'], axis=1))
rs =pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': predict
})

In [15]:
#훈련 데이터로 훈련세트와 테스트세트 나누기(Score값 측정용)
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(train.drop(['Survived'], axis=1), train['Survived'], random_state=42)

In [16]:
#Score값 측정
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.8098802395209581
0.8116591928251121


In [ ]:
print(rs)
rs.to_csv('result1.csv', index=False) 

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


In [ ]:
#결과 데이터 프레임 확인
#train.drop(['Survived', 'Age', 'Parch', 'Fare','Family'], axis=1).head()
#test.head()
#train.describe()

,Sex,SibSp,Pclass_3,Pclass_2,Pclass_1,Solo
0,0,1,True,False,False,False
1,1,1,False,False,True,False
2,1,0,True,False,False,True
3,1,1,False,False,True,False
4,0,0,True,False,False,True
